In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import env
from pydataset import data
import acquire
import prepare

### 1. Attendance Data
#### Read the data from the attendance table and calculate an attendance percentage for each student. One half day is worth 50% of a full day, and 10 tardies is equal to one absence.

In [2]:
attend = pd.read_sql('''
                    SELECT * FROM attendance
                    ''', env.conn('tidy_data'))
attend.head()

,Unnamed: 0,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08
0,Sally,P,T,T,H,P,A,T,T
1,Jane,A,P,T,T,T,T,A,T
2,Billy,A,T,A,A,H,T,P,T
3,John,P,T,H,P,P,T,P,P


In [3]:
attend = attend.rename(columns={'Unnamed: 0': 'name'})

In [20]:
attendance = attend.melt(id_vars = {'name'},
               var_name = 'date',
               value_name = 'attend')
attendance

,name,date,attend
0,Sally,2018-01-01,P
1,Jane,2018-01-01,A
2,Billy,2018-01-01,A
3,John,2018-01-01,P
4,Sally,2018-01-02,T
5,Jane,2018-01-02,P
6,Billy,2018-01-02,T
7,John,2018-01-02,T
8,Sally,2018-01-03,T
9,Jane,2018-01-03,T


In [18]:
#another way
at2 = attendance
at2.attend = at2.attend.map({'P':1, 'H':.5, 'A':0, 'T':.9})
at2.groupby('name').mean()

,attend
name,
Billy,0.5250
Jane,0.6875
John,0.9125
Sally,0.7625


In [21]:
attendance = attendance.pivot_table(index='name',
                      columns='attend',
                      values='date',
                      aggfunc='count')
attendance

attend,A,H,P,T
name,,,,
Billy,3.0,1.0,1.0,3.0
Jane,2.0,NaN,1.0,5.0
John,NaN,1.0,5.0,2.0
Sally,1.0,1.0,2.0,4.0


In [22]:
attendance = attendance.fillna(0.0)
attendance

attend,A,H,P,T
name,,,,
Billy,3.0,1.0,1.0,3.0
Jane,2.0,0.0,1.0,5.0
John,0.0,1.0,5.0,2.0
Sally,1.0,1.0,2.0,4.0


In [23]:
attendance.dtypes

attend
A    float64
H    float64
P    float64
T    float64
dtype: object

In [24]:
attendance['present'] = (attendance.P + attendance.H*.5 + attendance['T']*.9)/8
attendance

attend,A,H,P,T,present
name,,,,,
Billy,3.0,1.0,1.0,3.0,0.5250
Jane,2.0,0.0,1.0,5.0,0.6875
John,0.0,1.0,5.0,2.0,0.9125
Sally,1.0,1.0,2.0,4.0,0.7625


In [25]:
attendance.present

name
Billy    0.5250
Jane     0.6875
John     0.9125
Sally    0.7625
Name: present, dtype: float64

### 2. Coffee Levels
#### 2.a Read the coffee_levels table.

In [26]:
coffee = pd.read_sql('''
                    SELECT * FROM coffee_levels
                    ''', env.conn('tidy_data'))
coffee.head()

,hour,coffee_carafe,coffee_amount
0,8,x,0.816164
1,9,x,0.451018
2,10,x,0.843279
3,11,x,0.335533
4,12,x,0.898291


#### 2.b Transform the data so that each carafe is in it's own column.

In [27]:
coffee = coffee.pivot_table(index='hour',
                  columns='coffee_carafe',
                  values='coffee_amount').reset_index()

#### 2.c Is this the best shape for the data?

In [31]:
coffee
#this is easier to read but not considered 'tidy'. The original structure of coffee was 'tidy'.

coffee_carafe,hour,x,y,z
0,8,0.816164,0.189297,0.999264
1,9,0.451018,0.521502,0.915990
2,10,0.843279,0.023163,0.144928
3,11,0.335533,0.235529,0.311495
4,12,0.898291,0.017009,0.771947
5,13,0.310711,0.997464,0.398520
6,14,0.507288,0.058361,0.864464
7,15,0.215043,0.144644,0.436364
8,16,0.183891,0.544676,0.280621
9,17,0.391560,0.594126,0.436677


### 3. Cake Recipes
#### 3.a Read the cake_recipes table. This data set contains cake tastiness scores for combinations of different recipes, oven rack positions, and oven temperatures.

In [35]:
cake = pd.read_sql('''
                    SELECT * FROM cake_recipes
                    ''', env.conn('tidy_data'))
cake

,recipe:position,225,250,275,300
0,a:bottom,61.738655,53.912627,74.414730,98.786784
1,a:top,51.709751,52.009735,68.576858,50.228470
2,b:bottom,57.095320,61.904369,61.196980,99.248541
3,b:top,82.455004,95.224151,98.594881,58.169349
4,c:bottom,96.470207,52.001358,92.893227,65.473084
5,c:top,71.306308,82.795477,92.098049,53.960273
6,d:bottom,52.799753,58.670419,51.747686,56.183110
7,d:top,96.873178,76.101363,59.571620,50.971626


#### 3.b Tidy the data as necessary.

In [36]:
cake = cake.melt(id_vars='recipe:position',
         var_name='temp',
         value_name='score')
cake

,recipe:position,temp,score
0,a:bottom,225,61.738655
1,a:top,225,51.709751
2,b:bottom,225,57.095320
3,b:top,225,82.455004
4,c:bottom,225,96.470207
5,c:top,225,71.306308
6,d:bottom,225,52.799753
7,d:top,225,96.873178
8,a:bottom,250,53.912627
9,a:top,250,52.009735


In [37]:
cake[['recipe','position']] = cake['recipe:position'].str.split(':', expand = True)
cake = cake.drop(columns='recipe:position')
cake

,temp,score,recipe,position
0,225,61.738655,a,bottom
1,225,51.709751,a,top
2,225,57.095320,b,bottom
3,225,82.455004,b,top
4,225,96.470207,c,bottom
5,225,71.306308,c,top
6,225,52.799753,d,bottom
7,225,96.873178,d,top
8,250,53.912627,a,bottom
9,250,52.009735,a,top


In [38]:
# cake.pivot_table(index=['temp','recipe'],
#                 columns='position',
#                 values = 'score').reset_index()

#### 3.c Which recipe, on average, is the best? recipe b

In [44]:
cake.groupby('recipe').agg('mean').idxmax()

score    b
dtype: object

#### 3.d Which oven temperature, on average, produces the best results? 275

In [45]:
cake.groupby('temp').agg('mean').idxmax()

score    275
dtype: object

#### 3.e Which combination of recipe, rack position, and temperature gives the best result? recipe b, bottom rack, 300 degrees

In [46]:
cake[cake.score == cake.score.max()]

,temp,score,recipe,position
26,300,99.248541,b,bottom


### 4. Bonus: explore the other tables in the tidy_data database and reshape them as necessary so that they are in a tidy format.

#### Used the melted table below:

In [47]:
melted = pd.read_sql('''
                    SELECT * FROM melted
                    ''', env.conn('tidy_data'))
melted.head()

,date,variable,value
0,2000-01-03,A,-0.456694
1,2000-01-03,B,-0.446287
2,2000-01-03,C,-0.839698
3,2000-01-03,D,0.963339
4,2000-01-04,A,-1.758507


In [48]:
melted.pivot_table(index='date',
                  columns='variable',
                  values='value').reset_index()

variable,date,A,B,C,D
0,2000-01-03,-0.456694,-0.446287,-0.839698,0.963339
1,2000-01-04,-1.758507,-0.585874,1.824410,0.247688
2,2000-01-05,-1.233451,0.483213,0.928141,-0.547720
3,2000-01-06,1.085606,-0.483495,-1.897256,0.016627
4,2000-01-07,2.219914,1.195858,-0.506973,-0.498261
5,2000-01-10,0.160460,1.230199,-0.431519,1.806125
6,2000-01-11,0.448520,1.273968,-1.288276,-1.033904
7,2000-01-12,-1.507058,0.943487,-1.531071,0.182235
8,2000-01-13,1.116433,0.817832,0.020780,-0.630335
9,2000-01-14,-0.187418,1.115411,-0.357618,0.774023


#### Used the gapminder1 table below:

In [49]:
gap = pd.read_sql('''
                    SELECT * FROM gapminder1
                    ''', env.conn('tidy_data'))
gap.head()

,year,country,measure,measurement
0,1955,Afghanistan,pop,8891209.0
1,1960,Afghanistan,pop,9829450.0
2,1965,Afghanistan,pop,10997885.0
3,1970,Afghanistan,pop,12430623.0
4,1975,Afghanistan,pop,14132019.0


In [50]:
gap = gap.pivot_table(index=['year','country'],
               columns='measure',
               values='measurement').reset_index()
gap

measure,year,country,fertility,life_expect,pop
0,1955,Afghanistan,7.7000,30.332,8891209.0
1,1955,Argentina,3.1265,64.399,18927821.0
2,1955,Aruba,5.1500,64.381,53865.0
3,1955,Australia,3.4060,70.330,9277087.0
4,1955,Austria,2.5200,67.480,6946885.0
...,...,...,...,...,...
688,2005,Switzerland,1.4200,81.701,7489370.0
689,2005,Turkey,2.1430,71.777,69660559.0
690,2005,United Kingdom,1.8150,79.425,60441457.0
691,2005,United States,2.0540,78.242,295734134.0


In [51]:
# country with highest avg life_expectancy
gap.groupby('country').life_expect.agg('mean').idxmax()

'Iceland'